In [ ]:
# !pip install tensorflow
! https://www.kaggle.com/datasets/chetankv/dogs-cats-images

/bin/bash: line 1: https://www.kaggle.com/datasets/chetankv/dogs-cats-images: No such file or directory


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d chetankv/dogs-cats-images

Dataset URL: https://www.kaggle.com/datasets/chetankv/dogs-cats-images
License(s): CC0-1.0
dogs-cats-images.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-cats-images.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


### Data Preprocessing

In [ ]:
train_dataset = ImageDataGenerator(rescale=1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datset = ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_gen = train_dataset.flow_from_directory('dataset/training_set',
                                            target_size=(64,64),
                                            batch_size=32,
                                            class_mode='binary')


Found 8000 images belonging to 2 classes.


In [ ]:
val_gem = test_datset.flow_from_directory('dataset/test_set',
                                            target_size=(64,64),
                                            batch_size=32,
                                            class_mode='binary')

Found 2000 images belonging to 2 classes.


**Building of Model**

In [ ]:
cnn = tf.keras.models.Sequential()

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3]))

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [ ]:
# flatten
cnn.add(tf.keras.layers.Flatten())

In [ ]:
# fully connected layers
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [ ]:
from tensorflow.keras.optimizers.legacy import Adam

**Compile the model**

In [ ]:
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 6272)              0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [ ]:
history = cnn.fit(x=train_gen,validation_data=val_gem,epochs= 5)

Epoch 1/5
250/250 [==============================] - 61s 240ms/step - loss: 0.6689 - accuracy: 0.5878 - val_loss: 0.5975 - val_accuracy: 0.6800
Epoch 2/5
250/250 [==============================] - 55s 221ms/step - loss: 0.5951 - accuracy: 0.6784 - val_loss: 0.5515 - val_accuracy: 0.7250
Epoch 3/5
250/250 [==============================] - 53s 213ms/step - loss: 0.5432 - accuracy: 0.7216 - val_loss: 0.5267 - val_accuracy: 0.7455
Epoch 4/5
250/250 [==============================] - 54s 217ms/step - loss: 0.5106 - accuracy: 0.7465 - val_loss: 0.4867 - val_accuracy: 0.7760
Epoch 5/5
250/250 [==============================] - 55s 218ms/step - loss: 0.4812 - accuracy: 0.7695 - val_loss: 0.5158 - val_accuracy: 0.7515


In [ ]:
import numpy as np
from keras.preprocessing import image

In [ ]:
test_image = image.load_img('dataset/test_set/cats/cat.4001.jpg',target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = cnn.predict(test_image)
if result[0][0] == 1:
  print('dog')
else:
  print('cat')

1/1 [==============================] - 0s 108ms/step
cat
